# Unified Neural Network Training

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tum-pbs/PhiML/blob/main/docs/Networks.ipynb)
&nbsp; • &nbsp; [🌐 **Φ<sub>ML</sub>**](https://github.com/tum-pbs/PhiML)
&nbsp; • &nbsp; [📖 **Documentation**](https://tum-pbs.github.io/PhiML/)
&nbsp; • &nbsp; [🔗 **API**](https://tum-pbs.github.io/PhiML/phiml)
&nbsp; • &nbsp; [**▶ Videos**]()
&nbsp; • &nbsp; [<img src="images/colab_logo_small.png" height=4>](https://colab.research.google.com/github/tum-pbs/PhiML/blob/main/docs/Examples.ipynb) [**Examples**](https://tum-pbs.github.io/PhiML/Examples.html)

Φ<sub>ML</sub> provides basic tools to set up and train neural networks.
Users can choose one of several standard network architectures and configure them in one line of code.
After defining a loss function, they can then be trained in a unified way or using library-specific functions.

In [ ]:
%%capture
!pip install phiml

from matplotlib import pyplot as plt

## Network Architectures

The [neural network API](phiml/nn) includes multiple common network architectures, such as [multi-layer perceptrons](phiml/nn#phiml.nn.mlp),
[U-Nets](phiml/nn#phiml.nn.u_net), [convolutional networks](phiml/nn#phiml.nn.conv_net), [ResNets](phiml/nn#phiml.nn.res_net), [invertible networks](phiml/nn#phiml.nn.invertible_net), and [VGG-like networks](phiml/nn#phiml.nn.conv_classifier).
All networks operating on grids are supported in 1D, 2D and 3D for periodic and non-periodic domains.

In [ ]:
from phiml import math, nn
math.use('torch')

## Training an MLP

Let's create an MLP and train it to learn a 1D sine function.
First, we set up the network with two hidden layers of 64 neurons each.

In [ ]:
mlp = nn.mlp(in_channels=1, out_channels=1, layers=[64, 64], activation='ReLU')
mlp

Note that this is a standard neural network of the chosen library (a PyTorch Module in this case).
This allows you to quickly try many network designs without writing much code, even if you don't use Φ<sub>ML</sub> for anything else.

Next, let's generate some training data.

In [ ]:
DATASET = math.batch(examples=100)
x = math.random_uniform(DATASET, low=-math.PI, high=math.PI)
y = math.sin(x) + math.random_normal(DATASET) * .1
plt.scatter(x.numpy(), y.numpy())

For the loss function, we will use the $L^2$ comparing prediction to labels.
We use [`math.native_call()`](phiml/math#phiml.math.native_call) to call the network with native tensors reshaped to the preferences of the current backend library.
This will pass tensors of shape BCS to PyTorch and BSC to TensorFlow and Jax, where B denotes the packed batch dimensions, C the packed channel dimensions and S all spatial dimensions.

In [ ]:
def loss_function(network_input, label):
    prediction = math.native_call(mlp, network_input)
    return math.l2_loss(prediction - label), prediction

Next, we create an optimizer, such as [`SGD`](phiml/nn#phiml.nn.sgd), [`Adam`](phiml/nn#phiml.nn.adam), [`AdaGrad`](phiml/nn#phiml.nn.adagrad), or [`RMSprop`](phiml/nn#phiml.nn.rmsprop).
The returned optimizer is again a library-specific type (in this case a PyTorch optimizer).

In [ ]:
optimizer = nn.adam(mlp, learning_rate=5e-3)
optimizer

Finally, we repeatedly update the weights using backprop.
Here, we use [`update_weights()`](phiml/nn#phiml.nn.update_weights), passing in the network, optimizer, loss function, and loss function arguments.
If you want more control over the training loop, you can use library-specific functions as well, since the network and optimizer do not depend on Φ<sub>ML</sub>.

In [ ]:
for i in range(200):
    loss, pred = nn.update_weights(mlp, optimizer, loss_function, x, y)
plt.scatter(x.numpy(), pred.numpy())

## Training a U-Net

Following the principles of Φ<sub>ML</sub>, U-Nets, as well as many other network architectures, can [operate on 1D, 2D or 3D](N_Dimensional.html) grids.
The number of spatial dimensions, `in_spatial`, must be specified when creating the network.

In [ ]:
u_net = nn.u_net(in_channels=1, out_channels=1, levels=4, periodic=False, in_spatial=2)
optimizer = nn.adam(u_net, 1e-3)

Let's train the network to turn a 2D grid of random noise into a Gaussian distribution.

In [ ]:
SPATIAL = math.spatial(x=64, y=64)
x = math.random_normal(SPATIAL)
y = math.exp(-.5 * math.vec_squared((math.meshgrid(SPATIAL) / SPATIAL) - .5))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
ax1.imshow(x.numpy('y,x'))
ax2.imshow(y.numpy('y,x'))

Let's train the network!

In [ ]:
def loss_function():
    prediction = math.native_call(u_net, x)
    return math.l2_loss(prediction - y), prediction

for i in range(200):
    loss, pred = nn.update_weights(u_net, optimizer, loss_function)

plt.imshow(pred.numpy('y,x'))

## Further Reading

[🌐 **Φ<sub>ML</sub>**](https://github.com/tum-pbs/PhiML)
&nbsp; • &nbsp; [📖 **Documentation**](https://tum-pbs.github.io/PhiML/)
&nbsp; • &nbsp; [🔗 **API**](https://tum-pbs.github.io/PhiML/phiml)
&nbsp; • &nbsp; [**▶ Videos**]()
&nbsp; • &nbsp; [<img src="images/colab_logo_small.png" height=4>](https://colab.research.google.com/github/tum-pbs/PhiML/blob/main/docs/Examples.ipynb) [**Examples**](https://tum-pbs.github.io/PhiML/Examples.html)